In [1]:
# Importieren Sie die erforderlichen Bibliotheken
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from ucimlrepo import fetch_ucirepo
from prophet import Prophet
import matplotlib.pyplot as plt

# Fetch dataset
air_quality = fetch_ucirepo(id=360)

# Daten (als pandas Dataframes)
X = air_quality.data.features

# Kombinieren Sie Datum und Uhrzeit in eine einzelne DateTime-Spalte -> nur 14 Funktionen
X['DateTime'] = pd.to_datetime(X['Date'] + ' ' + X['Time'])  # Format (JJJJ-MM-DD HH:MM:SS, z. B.: 2004-03-10 18:00:00)
X = X.set_index('DateTime')
X.drop(['Date', 'Time'], axis=1, inplace=True)  # Entfernen Sie die Originaldatum- und Uhrzeit-Spalten
X.replace(-200, np.nan, inplace=True)
X.replace(-200.0, np.nan, inplace=True)
pd.set_option('display.width', 1000)

# Invertiere PT08.S3(NOx)
X['PT08.S3(NOx)'] = -1 * X['PT08.S3(NOx)']  # Vorzeichen umkehren

# Initialisieren Sie den Min-Max-Scaler
scaler = MinMaxScaler()

# Wählen Sie nur die zu skalierenden Spalten aus
features_to_scale = X.columns  # Da 'DateTime' ein Index ist, wird es nicht eingeschlossen

# Wenden Sie den Scaler auf die Features an
X_scaled = pd.DataFrame(scaler.fit_transform(X[features_to_scale]), columns=features_to_scale, index=X.index)

# Jetzt enthält X_scaled die normalisierten Daten, wobei 'DateTime' der Index ist
# Ändert nichts für lineare Regression -> Vorwärtsfüllen von NA-Werten
X_scaled.fillna(method='ffill', inplace=True)

# Visualisieren Sie die Daten mit Prophet
prophet_df = X_scaled.reset_index()[['DateTime', 'CO(GT)']]  # Angenommen, 'DateTime' ist Ihr Index und 'CO(GT)' ist Ihre Zielvariable

# Spalten in 'ds' und 'y' umbenennen
prophet_df.columns = ['ds', 'y']

# Prophet-Modell initialisieren
prophet_model = Prophet()

# Modell anpassen
prophet_model.fit(prophet_df)

# DataFrame für die Zukunft mit 14 Tagen erstellen
future = prophet_model.make_future_dataframe(periods=14, freq='D')

# Prognose für die Zukunft erstellen
forecast = prophet_model.predict(future)

# Nur den Trend und die Konfidenzintervalle für die nächsten 14 Tage anzeigen
forecast_14_days = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][-14:]

# Diagramm für die Zukunft erstellen und Unsicherheiten nicht anzeigen
fig = prophet_model.plot(forecast_14_days, xlabel='Date', ylabel='Forecast Value', figsize=(10, 6), uncertainty=False)

# Diagramm anzeigen
plt.title('Future Forecast for the Next 14 Days with Trend and Confidence Intervals')
plt.show()